In [1]:
## mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install snntorch segmentation-models-pytorch

In [3]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.cuda.amp import GradScaler, autocast
from pathlib import Path
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
from torchvision import models
import snntorch as snn
from snntorch import surrogate
from snntorch import utils
import gc

# ==========================================
# 1. CONFIGURATION
# ==========================================
CONFIG = {
    "base_dir": "/content/drive/MyDrive/GlacierHack_practice/Train",
    "project_dir": "/content/drive/MyDrive/Glacier_SNN_ResNet34_Unified",

    # HYPERPARAMETERS
    "time_steps": 8,
    "batch_size": 2,
    "lr": 1e-4,
    "epochs": 40,
    "beta": 0.9,
    "threshold": 0.5,
    "slope": 25,           # Steep gradient for SNN

    "num_workers": 2,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

os.makedirs(CONFIG['project_dir'], exist_ok=True)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

set_seed(42)

# ==========================================
# 2. DATASET
# ==========================================
class GlacierDataset(Dataset):
    def __init__(self, base_dir, transform=None):
        self.base_dir = Path(base_dir)
        self.band_dirs = [self.base_dir / f"Band{i}" for i in range(1, 6)]
        self.label_dir = self.base_dir / "labels"
        self.ids = sorted([p.stem for p in self.band_dirs[0].glob("*.tif")])
        self.transform = transform

    def __len__(self): return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        bands = [cv2.imread(str(d / f"{img_id}.tif"), cv2.IMREAD_UNCHANGED).astype(np.float32) for d in self.band_dirs]
        image = np.stack(bands, axis=-1)
        label = cv2.imread(str(self.label_dir / f"{img_id}.tif"), cv2.IMREAD_UNCHANGED)
        if label.ndim == 3: label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)

        p02, p98 = np.percentile(image, 2), np.percentile(image, 98)
        image = np.clip(image, p02, p98)
        image = (image - image.min()) / (image.max() - image.min() + 1e-6)

        mask = np.zeros_like(label, dtype=np.int64)
        mask[label == 85] = 1; mask[label == 170] = 2; mask[label == 255] = 3

        if self.transform:
            aug = self.transform(image=image, mask=mask)
            return aug["image"].float(), aug["mask"].long()
        return torch.tensor(image.transpose(2,0,1)).float(), torch.tensor(mask).long()

class Wrapper(Dataset):
    def __init__(self, ds, t): self.ds, self.t = ds, t
    def __len__(self): return len(self.ds)
    def __getitem__(self, i):
        img, mask = self.ds[i]
        img = img.numpy().transpose(1,2,0); mask = mask.numpy()
        res = self.t(image=img, mask=mask)
        return res['image'], res['mask'].long()

# AUGMENTATION (Fixed)
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5), A.RandomRotate90(p=0.5),
    A.GridDistortion(p=0.3), # Removed CoarseDropout to fix error
    ToTensorV2(),
])
val_transform = A.Compose([ToTensorV2()])

full_ds = GlacierDataset(CONFIG['base_dir'], transform=train_transform)
val_len = int(len(full_ds)*0.2)
train_ds, val_ds = random_split(full_ds, [len(full_ds)-val_len, val_len], generator=torch.Generator().manual_seed(42))
val_ds.dataset.transform = val_transform

train_loader = DataLoader(Wrapper(train_ds, train_transform), batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(Wrapper(val_ds, val_transform), batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)

# ==========================================
# 3. ARCHITECTURE: ResNet34 U-Net
# ==========================================
class ResNet34Encoder(nn.Module):
    def __init__(self, mode="CNN"):
        super().__init__()
        resnet = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)

        self.conv1 = nn.Conv2d(5, 64, kernel_size=7, stride=2, padding=3, bias=False)
        with torch.no_grad():
            self.conv1.weight[:, :3] = resnet.conv1.weight
            self.conv1.weight[:, 3:] = resnet.conv1.weight[:, :2]
        self.bn1 = resnet.bn1

        if mode == "SNN":
            self.relu = snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=surrogate.fast_sigmoid(slope=CONFIG['slope']), init_hidden=True)
        else:
            self.relu = resnet.relu
        self.maxpool = resnet.maxpool

        self.layer1 = self._convert(resnet.layer1, mode)
        self.layer2 = self._convert(resnet.layer2, mode)
        self.layer3 = self._convert(resnet.layer3, mode)
        self.layer4 = self._convert(resnet.layer4, mode)

    def _convert(self, block, mode):
        if mode == "CNN": return block
        layers = []
        for b in block:
            b.relu = snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'],
                               spike_grad=surrogate.fast_sigmoid(slope=CONFIG['slope']), init_hidden=True)
            layers.append(b)
        return nn.Sequential(*layers)

    def forward(self, x):
        feats = []
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        feats.append(x) # x0
        x = self.maxpool(x)
        x = self.layer1(x)
        feats.append(x) # x1
        x = self.layer2(x)
        feats.append(x) # x2
        x = self.layer3(x)
        feats.append(x) # x3
        x = self.layer4(x)
        feats.append(x) # x4
        return feats

class UnifiedDecoder(nn.Module):
    def __init__(self, mode="CNN"):
        super().__init__()
        spike_grad = surrogate.fast_sigmoid(slope=CONFIG['slope'])

        def block(in_c, out_c):
            act = snn.Leaky(beta=CONFIG['beta'], threshold=CONFIG['threshold'], spike_grad=spike_grad, init_hidden=True) if mode == "SNN" else nn.ReLU(inplace=True)
            return nn.Sequential(nn.Conv2d(in_c, out_c, 3, padding=1), nn.BatchNorm2d(out_c), act)

        self.up4 = nn.ConvTranspose2d(512, 256, 2, 2)
        self.dec4 = block(256+256, 256)
        self.up3 = nn.ConvTranspose2d(256, 128, 2, 2)
        self.dec3 = block(128+128, 128)
        self.up2 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec2 = block(64+64, 64)
        self.up1 = nn.ConvTranspose2d(64, 64, 2, 2)
        self.dec1 = block(64+64, 64)
        self.final_up = nn.ConvTranspose2d(64, 32, 2, 2)
        self.dec_final = block(32, 32)
        self.final = nn.Conv2d(32, 4, 1)

    def forward(self, enc_feats):
        x4, x3, x2, x1, x0 = enc_feats[4], enc_feats[3], enc_feats[2], enc_feats[1], enc_feats[0]
        u4 = self.up4(x4); d4 = self.dec4(torch.cat([x3, u4], 1))
        u3 = self.up3(d4); d3 = self.dec3(torch.cat([x2, u3], 1))
        u2 = self.up2(d3); d2 = self.dec2(torch.cat([x1, u2], 1))
        u1 = self.up1(d2); d1 = self.dec1(torch.cat([x0, u1], 1))
        out = self.dec_final(self.final_up(d1))
        return self.final(out)

class UnifiedUNet(nn.Module):
    def __init__(self, mode="CNN"):
        super().__init__()
        self.mode = mode
        self.encoder = ResNet34Encoder(mode)
        self.decoder = UnifiedDecoder(mode)

    def forward(self, x):
        if self.mode == "SNN":
            # REMOVED internal reset to prevent graph issues
            spk_rec = []
            for step in range(CONFIG['time_steps']):
                enc_feats = self.encoder(x)
                out = self.decoder(enc_feats)
                spk_rec.append(out)
            return torch.stack(spk_rec).mean(0)
        else:
            enc_feats = self.encoder(x)
            return self.decoder(enc_feats)

# ==========================================
# 4. TRAINING ENGINE
# ==========================================
def manual_reset(model):
    """Recursively reset all SNN layers"""
    for m in model.modules():
        if hasattr(m, "reset_mem"): m.reset_mem()

def train_phase(phase_name):
    print(f"\n🚀 STARTING PHASE: {phase_name}")
    torch.cuda.empty_cache()
    gc.collect()

    model = UnifiedUNet(mode=phase_name).to(CONFIG['device'])
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'])
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-4, steps_per_epoch=len(train_loader), epochs=CONFIG['epochs'])

    weights = torch.tensor([0.2, 1.0, 1.0, 3.0]).to(CONFIG['device'])
    criterion = nn.CrossEntropyLoss(weight=weights)
    scaler = GradScaler()

    best_mcc = -1.0
    history = {'loss': [], 'mcc': []}

    for epoch in range(CONFIG['epochs']):
        model.train()
        run_loss = 0

        loop = tqdm(train_loader, desc=f"{phase_name} Ep {epoch+1}")
        for imgs, masks in loop:
            imgs, masks = imgs.to(CONFIG['device']), masks.to(CONFIG['device'])

            if phase_name == "SNN": manual_reset(model)

            optimizer.zero_grad()

            if phase_name == "CNN":
                with autocast():
                    out = model(imgs)
                    loss = criterion(out, masks)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                out = model(imgs)
                loss = criterion(out, masks)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

            scheduler.step()
            run_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        # Val
        model.eval()
        preds, targets = [], []
        sample_vis = None
        with torch.no_grad():
            for i, (imgs, masks) in enumerate(val_loader):
                imgs = imgs.to(CONFIG['device'])
                if phase_name == "SNN": manual_reset(model)

                if phase_name == "CNN":
                    with autocast(): out = model(imgs)
                else:
                    out = model(imgs)

                preds.append(out.argmax(1).cpu())
                targets.append(masks.cpu())

                if i==0: sample_vis = (imgs[0].cpu().numpy(), masks[0].cpu().numpy(), preds[-1][0].numpy())

        mcc = matthews_corrcoef(torch.cat(targets).numpy().flatten(), torch.cat(preds).numpy().flatten())
        history['mcc'].append(mcc)
        history['loss'].append(run_loss/len(train_loader))
        print(f"   ✅ Val MCC: {mcc:.4f}")

        # Save Plot
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 3, 1); plt.imshow(sample_vis[0][[3,2,1]].transpose(1,2,0)); plt.title("Input")
        plt.subplot(1, 3, 2); plt.imshow(sample_vis[1], cmap='nipy_spectral'); plt.title("GT")
        plt.subplot(1, 3, 3); plt.imshow(sample_vis[2], cmap='nipy_spectral'); plt.title("Pred")
        plt.savefig(f"{CONFIG['project_dir']}/{phase_name}_ep{epoch+1}.png"); plt.close()

        if mcc > best_mcc:
            best_mcc = mcc
            torch.save(model.state_dict(), f"{CONFIG['project_dir']}/best_{phase_name}_ResNet34.pth")

    print(f"🏁 {phase_name} Finished. Best MCC: {best_mcc:.4f}")

# ==========================================
# 5. EXECUTE BOTH PHASES
# ==========================================
if __name__ == "__main__":
    train_phase("CNN")
    print("\n⏳ Switching to SNN Mode in 5 seconds...")
    import time; time.sleep(5)
    train_phase("SNN")


🚀 STARTING PHASE: CNN


/tmp/ipython-input-3539062809.py:229: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
CNN Ep 1:   0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipython-input-3539062809.py:247: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
CNN Ep 1: 100%|██████████| 10/10 [00:06<00:00,  1.57it/s, loss=1.61]
/tmp/ipython-input-3539062809.py:274: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): out = model(imgs)


   ✅ Val MCC: -0.0014


CNN Ep 2:   0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipython-input-3539062809.py:247: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
CNN Ep 2: 100%|██████████| 10/10 [00:04<00:00,  2.47it/s, loss=1.57]
/tmp/ipython-input-3539062809.py:274: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): out = model(imgs)


   ✅ Val MCC: -0.0212


...

   ✅ Val MCC: 0.4287


SNN Ep 39: 100%|██████████| 10/10 [00:12<00:00,  1.23s/it, loss=0.782]


   ✅ Val MCC: 0.4693


SNN Ep 40: 100%|██████████| 10/10 [00:12<00:00,  1.27s/it, loss=0.811]


   ✅ Val MCC: 0.4581
🏁 SNN Finished. Best MCC: 0.4991
